## Packages & Jeu de données

In [1]:
import pandas as pd

# Chargement du jeu de données 
full_data = pd.read_csv('Data/TRAIN_FULL/BAAC_2012_to_2022_V2_to_V4.csv', low_memory=False)

full_data.head()

,place,catu,grav,sexe,trajet,an_nais,senc,catv,occutc,obs,...,surf,infra,situ,env1,secu1,secu2,secu3,motor,vma,id_usager
0,1,1,0,0,5,1939,0.0,7,0.0,0.0,...,1.0,0.0,1.0,0.0,1,8,8,NaN,NaN,NaN
1,10,3,0,1,5,2008,0.0,7,0.0,0.0,...,1.0,0.0,1.0,0.0,8,8,8,NaN,NaN,NaN
2,1,1,0,1,5,1994,0.0,7,0.0,0.0,...,1.0,0.0,1.0,99.0,8,8,8,NaN,NaN,NaN
3,2,2,0,1,9,1992,0.0,7,0.0,0.0,...,1.0,0.0,1.0,99.0,1,8,8,NaN,NaN,NaN
4,1,1,0,1,5,1976,0.0,33,0.0,0.0,...,1.0,0.0,1.0,99.0,2,8,8,NaN,NaN,NaN


In [2]:
full_data.shape

(1245607, 46)

In [3]:
def convert_float_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    
def convert_float_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    data_frame[col_name] = data_frame[col_name].astype('category')
    
def convert_str_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype('category')
    
def convert_str_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)

### LIEUX

Les fichiers lieux contiennent globalement des informations relatives aux voies empruntées allant de leur catégorie, à la vitesse maximale autorisée (pour certaines années), le nom de la route, etc.

#### catr

Cette variable est importante, car elle indique le type de voie empruntée durant l'accident. On soupçonne qu'une corrélation forte peut exister entre les voies à grande vitesse et les accidents graves. En effet, si un accident se produit sur ce genre de voie, il y a peu de chance que ces derniers soient à faible allure. Ainsi, si les accidents sont majoritairement réalisés sous haute vitesse, il est probable que les accidents soient par conséquent plus violents.

Au sein des nouvelles réglementations, on observe l'apparition d'une catégorie (reliée à la valeur **"*7*"**) correspondant aux routes de métropole urbaine. On décidera donc de construire les catégories selon les valeurs suivantes : [**1**, **2**, **3**, **4**, **5**, **6**, **7**, **9**].

In [4]:
# Converti en variable catégorielle
convert_float_to_cat(full_data, 'catr')

1245607
1245607


#### voie, v1, v2

- **voie** : il est difficile de composer avec cette variable tant l'hétérogénéité est importante dans les valeurs saisies. De plus, sa pertinence n'est pas démontrée quand on dispose d'ores et déjà de la latitude et de la longitude de l'accident. Cette variable sera donc naturellement supprimée.
- **v1** : latitude de l'accident. Combinée à la longitude, cela pourrait en tant que variable numérique être utile pour créer des zones à accidents graves ou moins graves (carrefour en ville où de faibles accidents se produisent fréquemment par exemple).
- **v2** : longitude de l'accident. Combinée à la latitude, cela pourrait en tant que variable numérique être utile pour créer des zones à accidents graves ou moins graves (carrefour en ville où de faibles accidents se produisent fréquemment par exemple).

Hélas, les données de voie, v1, v2 sont trop peu qualitatives pour être exploitées sans un travail important. Nous décidons de supprimer ces colonnes pour l'instant. Cependant, nous nous réservons le droit de les retravailler plus tard afin d'améliorer potentiellement le modèle.

In [5]:
# Suppression des colonnes voie, v1, v2 
full_data.drop(labels=['voie', 'v1', 'v2'], axis=1, inplace=True)

#### circ

Correspond aux différents régimes de ciruclation (sens unique, double sens, ...). On soupçonne un intérêt pour cette variable et nous décidons donc de la converser. On décidera que les valeurs n'apportant aucune informations complémentaires seront représentées par un **"*5*"**.

In [6]:
# Harmonise les données
full_data['circ'] = full_data['circ'].replace([-1, 0], pd.NA)
full_data = full_data.dropna(subset=['circ'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'circ')

1245607
1175850


#### nbv

Correspond au nombre de voies, on décide de retenir cette variable, car il est possible que des voies uniques puissent être à l'origine d'accidents graves (dépassement de ligne blanche et rencontre frontale avec un autre automobiliste), ou bien vecteur d'une réduction des accidents graves (la majorité des gens peuvent ne pas vouloir franchir la ligne blanche et donc limite les dépassements dangereux). 

En disposant du nombre de voies, on suppose que nous avons affaire à une variable numérique (moins il y en a, plus on observera certains comportements, et inversement).

In [7]:
# Harmonisation des données
replace_dict = {'0.0': pd.NA, '1.0': '1', '2.0': '2', '3.0': '3', '4.0': '4', '5.0': '5', '6.0': '6', '7.0': '7', '8.0': '8', '9.0': '9', '10.0': '10', '11.0': '11', '12.0' :'12', '13.0' : '13', '#ERREUR' : pd.NA, -1: pd.NA, '-1.0': pd.NA, ' -1' : pd.NA}

full_data['nbv'] = full_data['nbv'].replace(replace_dict)
full_data = full_data.dropna(subset=['nbv'])

# Converti en variable numérique
convert_str_to_int(full_data, 'nbv')

1175850
1101027


In [8]:
full_data['nbv'].dtype

dtype('int32')

#### vosp

On suppose qu'il n'y a pas nécessairement de lien en l'existence d'une voie réservée, surtout lorsque cette voie n'est pas implicitement impliquée. On décide donc ne pas aborder cette variable pour l'instant. On se réserve le droit de la traiter dans des travaux futurs à des fins d'amélioration de modèles.

In [9]:
# Suppression de la colonne vosp
full_data.drop(labels=['vosp'], axis=1, inplace=True)

#### prof

On soupçonne que le relief de la route puisse légèrement influencer la présence d'accident (manque de visibilité en sommet de côte) et nous conservons donc cette valeur sujette à délétion sous réserve qu'elle soit impertinente plus tard.

In [10]:
# Harmonisation des données
full_data['prof'] = full_data['prof'].replace([-1, 0], pd.NA)
full_data = full_data.dropna(subset=['prof'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'prof')

1101027
1067913


In [11]:
full_data['prof'].dtype

CategoricalDtype(categories=[1, 2, 3, 4], ordered=False, categories_dtype=int32)

#### pr, pr1

Les points de repère ne semblent pas présenter d'intérêt au vu de ce qu'ils représentent pour les prendre en compte. Ils risqueraient de venir complexifier le modèle inutilement.

In [12]:
# Suppression des colonnes pr, pr1
full_data.drop(labels=['pr', 'pr1'], axis=1, inplace=True)

#### plan

Le tracé du plan semble important notamment pour les valeurs indiquant des virages, ou encore un tracé en **"*S"** indiquant un manque de visibilité, et des potentielles pertes d'adhérence.

In [14]:
# Harmonisation des données
full_data['plan'] = full_data['plan'].replace([-1, 0], pd.NA)
full_data = full_data.dropna(subset=['plan'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'plan')

#### lartpc, larrout

On compte environ 42% de données où la largeur du terre-plein central, supposément en mètres, n'est pas indiquée. Ainsi, il est difficile de rendre cette donnée exploitable pour la construction d'un modèle prédictif. 

Pour ce qui est de la largeur de la route, on dénombre déjà 18% de valeurs manquantes, s'ajoute à cela des données peu qualitatives avec des valeurs en cm ou en m sans indication. De plus, certaines voies ont des rapports du simple au quintuple ce qui n'apporte pas une plus-value importante à l'analyse pour le modèle prédictif. On décide donc de supprimer cette deuxième variable.

In [15]:
# Suppression des colonnes lartpc, larrout
full_data.drop(labels=['lartpc', 'larrout'], axis=1, inplace=True)

#### surf

Il est évident que les conditions liées à la nature de la chaussée amènent davantage d'accidents. On pensera notamment à des sols humides responsables du phénomène d'aquaplaning ou encore du verglas... Ainsi, nous décidons de sélectionner cette variable même si une corrélation forte risque de se produire avec d'autres variables liée aux conditions climatiques comme **"atm"**.

In [16]:
# Harmonisation des données
full_data['surf'] = full_data['surf'].replace([-1, 0], pd.NA)
full_data = full_data.dropna(subset=['surf'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'surf')

1039263
1032091


In [17]:
full_data['surf'].dtype

CategoricalDtype(categories=[1, 2, 3, 4, 5, 6, 7, 8, 9], ordered=False, categories_dtype=int32)

#### infra

Correspond à l'aménagement de la route au lieu de l'accident. On voit peu d'intérêt à cette variable, car d'autres variables recensent d'ores et déjà, et avec plus de pertinence, des zones d'intérêts. On pensera notamment à **"catr"**, **"plan"** ou encore **"prof"**.

In [18]:
# Suppression de la colonne infra
full_data.drop(labels=['infra'], axis=1, inplace=True)

#### situ

La situation de l'accident (chaussée, trottoir, ...) permet d'avoir des informations intéressantes sur les circonstances de l'accident. En effet, si l'accident se produit sur un trottoir, on peut s'interroger sur la violence de l'accident pour déporter le véhicule jusqu'à l'intérieur du trottoir. Ainsi, nous décidons de retenir cette variable pour sa pertinence en assignat de manière catégorielle les différentes situations.

In [19]:
# Harmonisation des données
full_data['situ'] = full_data['situ'].replace([-1, 0], pd.NA)
full_data = full_data.dropna(subset=['situ'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'situ')

1032091
1012171


#### vma, env1

La **"vma"** ou **V**itesse **M**aximale **A**utorisée est un indicateur trés intéressant, cependant le type de route spécifié au sein de la variable **"catr"** sous-entend déjà les informations relatives à ces vitesses. Seules les modifications des réglementations de vitesse ne peuvent être lues. De plus, fort de ses 65% de valeurs manquantes, c'est une variable difficilement exploitable. Il serait plus simlple de l'assigner manuellement en fonction de **"catr"** ce qui la rendrait encore plus inutile.

Pour ce qui est de **"env1"** qui traduit la présence d'une école à proximité, cet indicateur pourrait être intéressant malheureusement, il comporte 34% de valeurs manquantes. Au vu de la plus-value de cette variable, il est jugé non nécessaire de le retenir. On se réserve le droit d'y avoir recours ultérieurement. 

In [20]:
# Suppression des colonnes vma, env1
full_data.drop(labels=['vma', 'env1'], axis=1, inplace=True)

In [21]:
full_data.to_csv('Data/TRAIN_FULL/BAAC_2012_to_2022_V3_to_V4.csv', index=False)

Désormais, toutes les variables propres aux usagers ont été traitées, on sauvegarde une version intermédiaire de ce fichier.